In [1]:
from torchvision import models as models
import torch
from torch import nn

In [2]:
resnet_test = models.__dict__['resnet18'](pretrained=False)

In [3]:
print(resnet_test)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [5]:
class BasicBlock(nn.Module):
    
    def __init__(self, insize, filters, downsample=None):
        super(BasicBlock, self).__init__()
                
        self.bn1 = nn.BatchNorm2d(insize)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels=insize, out_channels=filters, kernel_size=3, stride=1,
                     padding=1, groups=1, bias=False, dilation=1)
        self.bn2 = nn.BatchNorm2d(filters)
        self.conv2 = nn.Conv2d(in_channels=filters, out_channels=filters, kernel_size=3, stride=1,
                     padding=1, groups=1, bias=False, dilation=1)
        self.downsample = downsample
        
    def forward(self,x):
        identity = x
        
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        
        out = self.bn2(x)
        out = self.relu(out)
        out = self.conv2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
        
        # when we change size we need to downsample I guess?
        out += identity
        
        return out
        

In [10]:
class WResNet(nn.Module):

    def __init__(self, block, layers=[2,2,2,2], k=4, num_classes=1000):
        super(WResNet, self).__init__()
        
        self.bn1 = nn.BatchNorm2d(3) # norm variable in torchvision
        self.conv1 = nn.Conv2d(3, 16*k, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        #### first part
        
        self.group1 = self._make_layer(block, 16*k, 16*k, layers[0])
        self.group2 = self._make_layer(block, 16*k, 32*k, layers[1])
        
        #### second part
        
        self.group3 = self._make_layer(block, 32*k, 64*k, layers[2])
        self.group4 = self._make_layer(block, 64*k, 128*k, layers[3])

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128*k, num_classes)
            
    def _make_layer(self, block, in_planes, out_planes, num_blocks, stride=1, dilate=False):
        downsample = None
        
        downsample_conv = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1,
                     padding=1)
        downsample_norm = nn.BatchNorm2d(out_planes)
        
        if in_planes != out_planes:
            #downsample = nn.Sequential(
            #    conv1x1(in_planes, out_planes, stride),
            #    nn.BatchNorm2d(out_planes),
            #)
            downsample = nn.Sequential(
                downsample_conv,
                #conv1x1(in_planes, out_planes, stride),
                downsample_norm
            )
        
        layers = []
        layers.append(block(insize=in_planes, filters=out_planes, downsample=downsample))
        
        for _ in range(1, num_blocks):
            layers.append(block(insize=out_planes, filters=out_planes))
        
        return nn.Sequential(*layers)
        
    def forward(self, x):
        
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv1(x)
        # does changing order change this?
        x = self.maxpool(x)
        
        x = self.group1(x)
        x = self.group2(x)
        
        x = self.group3(x)
        x = self.group4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

In [15]:
model = WResNet(block=BasicBlock, k=10)

In [8]:
test = torch.randn((10,3,32,32))

In [16]:
model(test).shape

torch.Size([10, 1000])